In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the teams
Teams = [
    'Baylor', 'Iowa State', 'University of Kansas', 'Kansas State',
    'University of Oklahoma', 'Oklahoma State', 'Texas Christian',
    'University of Texas Austin', 'Texas Tech', 'West Virginia'
]

In [2]:
def create_scores_matrix(n=10):
    """Creates a random n×n matrix with:
       - zeros on diagonal
       - if i beats j, then matrix[i,j] = 1 and matrix[j,i] = -1
       - ensures all teams play against each other (round-robin)
    """
    scores = np.zeros((n, n))

    # For each pair of teams
    for i in range(n):
        for j in range(i+1, n):
            # Randomly decide winner
            if random.random() > 0.5:  # i wins
                scores[i, j] = 1
                scores[j, i] = -1
            else:  # j wins
                scores[i, j] = -1
                scores[j, i] = 1

    return scores

In [3]:
def create_differentials_matrix(scores, n=10):
    """Creates a random n×n matrix of score differentials
       that corresponds to the game results in scores matrix.
    """
    differentials = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if scores[i, j] == 1:  # If team i beat team j
                # Generate a random point differential between 1 and 28
                differentials[i, j] = np.random.randint(1, 29)
                differentials[j, i] = -differentials[i, j]

    return differentials

In [4]:
Scores = create_scores_matrix()
Differentials = create_differentials_matrix(Scores)

print("Scores Matrix:")
print(Scores)
print("\nDifferentials Matrix:")
print(Differentials)

Scores Matrix:
[[ 0.  1.  1. -1. -1. -1.  1. -1. -1.  1.]
 [-1.  0. -1. -1. -1.  1. -1.  1.  1.  1.]
 [-1.  1.  0.  1.  1.  1.  1.  1. -1. -1.]
 [ 1.  1. -1.  0.  1. -1. -1. -1. -1. -1.]
 [ 1.  1. -1. -1.  0.  1. -1.  1. -1. -1.]
 [ 1. -1. -1.  1. -1.  0. -1.  1. -1. -1.]
 [-1.  1. -1.  1.  1.  1.  0.  1. -1. -1.]
 [ 1. -1. -1.  1. -1. -1. -1.  0. -1.  1.]
 [ 1. -1.  1.  1.  1.  1.  1.  1.  0. -1.]
 [-1. -1.  1.  1.  1.  1.  1. -1.  1.  0.]]

Differentials Matrix:
[[  0.   5.   7. -16. -16. -27.  22. -26. -11.  16.]
 [ -5.   0. -19. -13. -17.  12. -15.  11.   9.   8.]
 [ -7.  19.   0.  22.  16.   5.  25.  10.  -8. -19.]
 [ 16.  13. -22.   0.   5. -13.  -8.  -7. -27. -26.]
 [ 16.  17. -16.  -5.   0.   5. -26.  27.  -3. -19.]
 [ 27. -12.  -5.  13.  -5.   0. -16.  26.  -7. -19.]
 [-22.  15. -25.   8.  26.  16.   0.  18. -14. -17.]
 [ 26. -11. -10.   7. -27. -26. -18.   0. -13.  22.]
 [ 11.  -9.   8.  27.   3.   7.  14.  13.   0. -19.]
 [-16.  -8.  19.  26.  19.  19.  17. -22.  19.   0.]]


# College Football Ranking Methods: Colley and Massey

## Colley's Method

Colley's method, developed by astrophysicist Wesley Colley, provides a bias-free ranking system based only on wins and losses.

### Key Features:

* **Laplace's Rule of Succession** replaces simple win percentages
* **Automatically accounts for strength of schedule**
* **Uses only wins and losses** (no point differentials)
* **Produces a well-conditioned system** that always has a unique solution

### Mathematical Framework:

1. **Basic Rating Formula**:
   * $r_i = \frac{w_i + 1}{t_i + 2}$
   * Where $r_i$ is team i's rating, $w_i$ is wins, and $t_i$ is total games

2. **Incorporating Strength of Schedule**:
   * $r_i = \frac{1 + \frac{w_i - l_i}{2}}{t_i + 1}$
   * Where $l_i$ is the number of losses

3. **Final System of Equations**:
   * $(2 + t_i)r_i - \sum_{j} r_j = 1 + \frac{w_i - l_i}{2}$
   * Where the sum is over all teams j that team i played against

4. **Matrix Representation**:
   * $\mathbf{C}\mathbf{r} = \mathbf{b}$
   * Where $\mathbf{C}$ is the Colley matrix with elements:
     * $C_{ii} = 2 + t_i$ (diagonal)
     * $C_{ij} = -n_{ij}$ (off-diagonal, $n_{ij}$ is number of games between teams i and j)
   * And $b_i = 1 + \frac{w_i - l_i}{2}$

5. **Properties**:
   * Ratings are bounded between 0 and 1
   * Average rating across all teams is exactly 0.5

## Massey's Method

Massey's method, developed by Kenneth Massey, ranks teams by incorporating point differentials to predict game outcomes.

### Key Features:

* **Uses point differentials** rather than just wins/losses
* **Creates an overdetermined system** (more equations than unknowns)
* **Solves using least squares optimization**
* **Minimizes prediction error** across all games

### Mathematical Framework:

1. **Basic Game Equation**:
   * $r_i - r_j = d_{ij}$
   * Where $d_{ij}$ is the point differential (team i's score minus team j's score)

2. **Matrix Representation**:
   * $\mathbf{P}\mathbf{r} = \mathbf{d}$
   * Where for each game:
     * A row in $\mathbf{P}$ has a 1 in position i, -1 in position j, and 0 elsewhere
     * The corresponding element in $\mathbf{d}$ is the point differential $d_{ij}$

3. **Least Squares Formulation**:
   * Minimize $||\mathbf{P}\mathbf{r} - \mathbf{d}||^2$
   * Leads to the normal equations: $\mathbf{P}^T\mathbf{P}\mathbf{r} = \mathbf{P}^T\mathbf{d}$

4. **Rank Deficiency Resolution**:
   * Replace the last row of $\mathbf{P}^T\mathbf{P}$ with all 1's
   * Replace the last element of $\mathbf{P}^T\mathbf{d}$ with 0
   * This enforces the constraint that ratings sum to zero: $\sum_i r_i = 0$

5. **Properties**:
   * Ratings represent team strength on a point scale
   * The difference between two teams' ratings predicts the point differential in a game between them

## Practical Implementations

* **Colley's Method**: Better for rankings where only wins and losses matter
* **Massey's Method**: Better for rankings where margin of victory is relevant
* **Combining Methods**: The BCS used both methods (along with others) for a more robust ranking system

In [5]:
def colley_method(scores):
    """Implements Colley's ranking method."""
    n = scores.shape[0]

    # Define variables
    games = np.abs(scores)
    total = np.sum(games, axis=1)

    # Construct Colley's matrix and the right-hand side vector
    colley_matrix = 2 * np.eye(n) + np.diag(total) - games
    right_side = 1 + 0.5 * np.sum(scores, axis=1)

    # Solve the linear system
    ranks_colley = np.linalg.solve(colley_matrix, right_side)

    return ranks_colley

In [6]:
def massey_method(differentials):
    """Implements Massey's ranking method."""
    n = differentials.shape[0]

    # Create P matrix and B vector
    P = []
    B = []

    # Loop through the upper triangular part of the Differentials matrix
    for j in range(n-1):
        for k in range(j+1, n):
            if differentials[j, k] != 0:
                row = np.zeros(n)
                row[j] = 1
                row[k] = -1
                P.append(row)
                B.append(differentials[j, k])

    # Convert lists to numpy arrays
    P = np.array(P)
    B = np.array(B)

    # Create the normal system of linear equations
    A = np.dot(P.T, P)
    D = np.dot(P.T, B)

    # Substitute the last row of the matrix and the last element of the vector
    A[-1, :] = np.ones(n)
    D[-1] = 0

    # Solve the system
    ranks_massey = np.linalg.solve(A, D)

    return ranks_massey

In [7]:
RanksColley = colley_method(Scores)
RanksMassey = massey_method(Differentials)

# Display Colley's rankings
print("\nColley's Rankings:")
colley_order = np.argsort(RanksColley)[::-1]
colley_ranks = RanksColley[colley_order]

for j in range(len(Teams)):
    print(f'{colley_ranks[j]:8.3f} {Teams[colley_order[j]]}')

# Display Massey's rankings
print("\nMassey's Rankings:")
massey_order = np.argsort(RanksMassey)[::-1]
massey_ranks = RanksMassey[massey_order]

for j in range(len(Teams)):
    print(f'{massey_ranks[j]:8.3f} {Teams[massey_order[j]]}')


Colley's Rankings:
   0.708 Texas Tech
   0.625 West Virginia
   0.625 University of Kansas
   0.542 Texas Christian
   0.458 Baylor
   0.458 University of Oklahoma
   0.458 Iowa State
   0.375 University of Texas Austin
   0.375 Oklahoma State
   0.375 Kansas State

Massey's Rankings:
   7.300 West Virginia
   6.300 University of Kansas
   5.500 Texas Tech
   0.500 Texas Christian
   0.200 Oklahoma State
  -0.400 University of Oklahoma
  -2.900 Iowa State
  -4.600 Baylor
  -5.000 University of Texas Austin
  -6.900 Kansas State


In [9]:
def analyze_colley_switch():
    """Switch results between top two teams in Colley ranking"""
    print("\n--- ANALYSIS: Switching Game Results between Top Two Teams in Colley Method ---")

    # Identify the current top two teams according to Colley's rankings
    top_teams_colley = colley_order[:2]
    print(f"Top two teams: {Teams[top_teams_colley[0]]} and {Teams[top_teams_colley[1]]}")

    # Get current result between these teams
    current_result = Scores[top_teams_colley[0], top_teams_colley[1]]
    print(f"Current result: {Teams[top_teams_colley[0]]} {'beat' if current_result == 1 else 'lost to'} {Teams[top_teams_colley[1]]}")

    # Store original scores for later restoration
    original_scores = Scores.copy()

    # Switch the result of the game between the top two teams
    Scores[top_teams_colley[0], top_teams_colley[1]] *= -1
    Scores[top_teams_colley[1], top_teams_colley[0]] *= -1

    print(f"Switched result: {Teams[top_teams_colley[0]]} {'beat' if Scores[top_teams_colley[0], top_teams_colley[1]] == 1 else 'lost to'} {Teams[top_teams_colley[1]]}")

    # Recompute Colley's rankings
    new_ranks_colley = colley_method(Scores)
    new_colley_order = np.argsort(new_ranks_colley)[::-1]

    print("\nUpdated Colley's Rankings After Game Result Switch:")
    for j in range(len(Teams)):
        print(f'{new_ranks_colley[new_colley_order[j]]:8.3f} {Teams[new_colley_order[j]]}')

    # Check if rankings changed
    if np.array_equal(colley_order[:2], new_colley_order[:2]):
        print("\nTop two teams' ranking did NOT change.")
    else:
        print("\nTop two teams' ranking CHANGED.")
        print(f"Previous top 2: {Teams[colley_order[0]]} and {Teams[colley_order[1]]}")
        print(f"New top 2: {Teams[new_colley_order[0]]} and {Teams[new_colley_order[1]]}")

    # Restore original matrix
    np.copyto(Scores, original_scores)

In [10]:
def analyze_massey_switch():
    """Switch results between top two teams in Massey ranking"""
    print("\n--- ANALYSIS: Switching Game Results between Top Two Teams in Massey Method ---")

    # Identify the current top two teams according to Massey's rankings
    top_teams_massey = massey_order[:2]
    print(f"Top two teams: {Teams[top_teams_massey[0]]} and {Teams[top_teams_massey[1]]}")

    # Get current result between these teams
    current_result = Differentials[top_teams_massey[0], top_teams_massey[1]]
    print(f"Current point differential: {Teams[top_teams_massey[0]]} {current_result} points {'ahead of' if current_result > 0 else 'behind'} {Teams[top_teams_massey[1]]}")

    # Store original data for later restoration
    original_differentials = Differentials.copy()
    original_scores = Scores.copy()

    # Switch the result of the game between the top two teams
    if Differentials[top_teams_massey[0], top_teams_massey[1]] != 0:
        Differentials[top_teams_massey[0], top_teams_massey[1]] *= -1
        Differentials[top_teams_massey[1], top_teams_massey[0]] *= -1

        # Also update the Scores matrix to be consistent
        Scores[top_teams_massey[0], top_teams_massey[1]] *= -1
        Scores[top_teams_massey[1], top_teams_massey[0]] *= -1

    print(f"Switched point differential: {Teams[top_teams_massey[0]]} {Differentials[top_teams_massey[0], top_teams_massey[1]]} points {'ahead of' if Differentials[top_teams_massey[0], top_teams_massey[1]] > 0 else 'behind'} {Teams[top_teams_massey[1]]}")

    # Recompute Massey's rankings
    new_ranks_massey = massey_method(Differentials)
    new_massey_order = np.argsort(new_ranks_massey)[::-1]

    print("\nUpdated Massey's Rankings After Game Result Switch:")
    for j in range(len(Teams)):
        print(f'{new_ranks_massey[new_massey_order[j]]:8.3f} {Teams[new_massey_order[j]]}')

    # Check if rankings changed
    if np.array_equal(massey_order[:2], new_massey_order[:2]):
        print("\nTop two teams' ranking did NOT change.")
    else:
        print("\nTop two teams' ranking CHANGED.")
        print(f"Previous top 2: {Teams[massey_order[0]]} and {Teams[massey_order[1]]}")
        print(f"New top 2: {Teams[new_massey_order[0]]} and {Teams[new_massey_order[1]]}")

    # Restore original matrices
    np.copyto(Differentials, original_differentials)
    np.copyto(Scores, original_scores)

In [11]:
def visualize_rankings():
    """Create a bar chart comparing Colley and Massey rankings"""
    plt.figure(figsize=(12, 8))

    # Normalize rankings to be comparable
    colley_normalized = (RanksColley - np.min(RanksColley)) / (np.max(RanksColley) - np.min(RanksColley))
    massey_normalized = (RanksMassey - np.min(RanksMassey)) / (np.max(RanksMassey) - np.min(RanksMassey))

    # Sort teams by average ranking
    avg_rank = (colley_normalized + massey_normalized) / 2
    team_order = np.argsort(avg_rank)[::-1]

    x = np.arange(len(Teams))
    width = 0.35

    plt.bar(x - width/2, colley_normalized[team_order], width, label='Colley Method')
    plt.bar(x + width/2, massey_normalized[team_order], width, label='Massey Method')

    plt.ylabel('Normalized Ranking Score')
    plt.title('Comparison of Colley and Massey Ranking Methods')
    plt.xticks(x, [Teams[i] for i in team_order], rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()

    plt.savefig('ranking_comparison.png')
    plt.close()
    print("\nRanking comparison chart saved as 'ranking_comparison.png'")

# Run the analyses
analyze_colley_switch()
analyze_massey_switch()
visualize_rankings()



--- ANALYSIS: Switching Game Results between Top Two Teams in Colley Method ---
Top two teams: Texas Tech and West Virginia
Current result: Texas Tech lost to West Virginia
Switched result: Texas Tech beat West Virginia

Updated Colley's Rankings After Game Result Switch:
   0.792 Texas Tech
   0.625 University of Kansas
   0.542 West Virginia
   0.542 Texas Christian
   0.458 University of Oklahoma
   0.458 Iowa State
   0.458 Baylor
   0.375 University of Texas Austin
   0.375 Oklahoma State
   0.375 Kansas State

Top two teams' ranking CHANGED.
Previous top 2: Texas Tech and West Virginia
New top 2: Texas Tech and University of Kansas

--- ANALYSIS: Switching Game Results between Top Two Teams in Massey Method ---
Top two teams: West Virginia and University of Kansas
Current point differential: West Virginia 19.0 points ahead of University of Kansas
Switched point differential: West Virginia -19.0 points behind University of Kansas

Updated Massey's Rankings After Game Result Switc

In [12]:
def compare_rankings():
    """Compare how teams are ranked differently between methods"""
    print("\n--- COMPARISON: Colley vs Massey Rankings ---")

    # Create a table of rankings
    print(f"{'Team':<25} {'Colley Rank':<15} {'Massey Rank':<15} {'Difference':<10}")
    print("-" * 65)

    for i, team in enumerate(Teams):
        colley_position = np.where(colley_order == i)[0][0] + 1  # +1 for 1-indexed ranking
        massey_position = np.where(massey_order == i)[0][0] + 1  # +1 for 1-indexed ranking
        diff = colley_position - massey_position

        print(f"{team:<25} {colley_position:<15} {massey_position:<15} {diff:+d}")

    # Calculate rank correlation
    from scipy.stats import spearmanr

    # Convert to rankings (1-indexed)
    colley_ranks_by_team = np.zeros(len(Teams))
    massey_ranks_by_team = np.zeros(len(Teams))

    for i in range(len(Teams)):
        colley_ranks_by_team[colley_order[i]] = i + 1
        massey_ranks_by_team[massey_order[i]] = i + 1

    corr, p = spearmanr(colley_ranks_by_team, massey_ranks_by_team)
    print(f"\nSpearman Rank Correlation: {corr:.4f} (p-value: {p:.4f})")

    if corr > 0.7:
        print("The rankings from both methods are strongly correlated.")
    elif corr > 0.3:
        print("The rankings from both methods have moderate correlation.")
    else:
        print("The rankings from both methods have weak correlation.")

compare_rankings()




--- COMPARISON: Colley vs Massey Rankings ---
Team                      Colley Rank     Massey Rank     Difference
-----------------------------------------------------------------
Baylor                    5               8               -3
Iowa State                7               7               +0
University of Kansas      3               2               +1
Kansas State              10              10              +0
University of Oklahoma    6               6               +0
Oklahoma State            9               5               +4
Texas Christian           4               4               +0
University of Texas Austin 8               9               -1
Texas Tech                1               3               -2
West Virginia             2               1               +1

Spearman Rank Correlation: 0.8061 (p-value: 0.0049)
The rankings from both methods are strongly correlated.
